In [4]:
import pandas as pd
import numpy as np
import torch
from data_reader import read_interaction_matrix
R = read_interaction_matrix()
from tqdm import tqdm

n_users, n_items = R.shape
n_latent = 64
r = 0.001

# learning rate
INIT_LR = 0.05
END_LR = 0.0001
n_epochs = 750

# error check steps
erc_step = 100
# min error threshold
eth = 1e-4

rng = np.random.default_rng(seed=1234)
scale = 1./n_latent
Q0 = rng.normal(scale=scale, size=(n_users, n_latent))
P0 = rng.normal(scale=scale, size=(n_items, n_latent))

def E(R, P, Q):
    return np.abs(R - Q@P.T).sum() + r*((Q*Q).sum() + (P*P).sum())

def LR(epoch, warmup):
    s = warmup / n_epochs
    x = (epoch - n_epochs)/n_epochs
    
    if epoch >= warmup:
        x = x - s
        y = ((1 + x))
        
        # y = (n_epochs*(s-1) + 1/(1 - s))*x + n_epochs*(1-s)
        # y = y * n_epochs / (n_epochs - s*n_epochs)
    else:
        x += 1
        y = (-2./s)*(x**2) + 2*s
        
    # f = lambda e : ((e/n_epochs)) * (((e/5 + (1-stop_warmup)*n_epochs)/n_epochs)**5)
    # y = f(2/3*n_epochs + (-n_epochs + epoch)*3.)
    # if epoch < stop_warmup:
    #     x *= ((-epoch + stop_warmup//2)*3/n_epochs)**3
    #     if epoch % 20 == 0:
    #         print("warmup")
    # z = lambda x : (x - 1/3)**(0.3) * np.exp(-x**(0.8))
    
    lr = INIT_LR - (INIT_LR-END_LR) * y
    return lr


In [15]:
from data_reader import read_interaction_matrix
R = read_interaction_matrix()

from mf import matrix_factorization
from matplotlib import pyplot as plt

def get_errors(logs):
    errors = [log[0] for log in logs]
    return errors

q, p, err, logs = matrix_factorization(R, lr=(1e-2,1e-4), n_epochs=500, reg=0.001, n_latent=64)
errors = get_errors(logs)
plt.plot(errors)

0.01 0.0001


2it [00:00, 13.41it/s]

err: 2.30e+04, lr: 0.0010899


109it [00:02, 57.16it/s]

err: 1.76e+04, lr: 0.0097044


...

err: 1.75e+04, lr: 0.0056659


1114it [00:18, 44.25it/s]

err: 1.75e+04, lr: 0.0053385


1181it [00:20, 59.01it/s]


KeyboardInterrupt: 

In [12]:
import numpy as np
S = (q @ p.T)
S.sum(), R.sum()

(22710.410707193598, 23038.0)

In [14]:
S[R.nonzero()]

array([0.14531313, 0.25270802, 0.15089348, ..., 0.20097695, 0.13145251,
       0.0401452 ])

In [14]:
from mf import error

print(error(R, q, p, 0.001))

22985.33488759431


In [7]:
S = (q@p.T)

def f(f):
    i = 5
    i += 1 if f < 0 else 0
    return str(f)[:i]

[f(x) for x in [S[0,6], S[0,7], S.mean(), S.min(), S.max()]]

['-0.013', '0.002', '0.023', '-0.492', '1.597']

In [8]:
S[R.nonzero()].mean(), (S - S * R).mean()

(0.4351097056356439, 0.012915999784935794)

In [4]:
from mf import init_pq

q, p = init_pq(*R.shape, 64)